In [1]:
import numpy as np
import pandas as pd

In [2]:
users = pd.read_csv('../../data/Users.csv', delimiter=';')
books = pd.read_csv('../../data/Books.csv', delimiter=';', dtype={'ISBN': str, 'Title': str, 'Author': str, 'Year': np.int16, 'Publisher': str})
ratings = pd.read_csv('../../data/Ratings.csv', delimiter=';', dtype={'User-ID': np.int32, 'ISBN': str, 'Rating': np.int8})

/var/folders/yn/16crnc792zg2yr78zf0qg0j40000gq/T/ipykernel_44573/2231409931.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv('../../data/Users.csv', delimiter=';')


In [3]:
import sys
sys.path.append('../..')

from src.evaluation import average_precision
from sklearn.model_selection import train_test_split

In [4]:
train_ratings, test_ratings = train_test_split(ratings, test_size=0.2, random_state=42)

In [5]:
import src.recommenders.user_collaborative_filtering as ubcf

model = ubcf.UserRecommender()
model.fit(users=users, items=books, ratings=train_ratings)

/Users/oleh.sannikov/Documents/Навчання/Науки про дані/RecSys/CP/rs-ucu-books-recommender/experiments/sunnycows/../../src/utils/data_preprocessing.py:60: RuntimeWarning: Mean of empty slice.
  means = np.array([normalized[i].data.mean() for i in range(normalized.shape[0])])
/Users/oleh.sannikov/miniconda3/envs/recsys/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [6]:
test_users = test_ratings['User-ID'].unique()[:1000]
len(test_users)

1000

In [7]:
best_items = model.predict(test_users, items=books)

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [04:33<00:00,  3.66it/s]


In [10]:
len(best_items)
best_items[38781]['ISBN']

57144    0671003593
73871    0671000403
57042    0373078358
31483    0425176940
4449     1551668912
Name: ISBN, dtype: object

In [11]:
from tqdm import tqdm

rating_mask = test_ratings['Rating'] >= 5

mean_ap = 0
for i in tqdm(range(len(test_users))):
    user_mask = test_ratings['User-ID'] == test_users[i]
    relevant_items = test_ratings.loc[(user_mask & rating_mask)].sort_values(by='Rating', ascending=False)
    if len(relevant_items) == 0:
        continue
    user_id = test_users[i]
    ap = average_precision(best_items[user_id]['ISBN'].to_numpy(), relevant_items['ISBN'].to_numpy(), k=5)
    mean_ap += ap

mean_ap /= len(test_users)
mean_ap

100%|██████████| 1000/1000 [00:02<00:00, 431.44it/s]


0.0061